Read EEG data   
source localized    
power would better to get neural info   

In [1]:
import scipy.io

mat = scipy.io.loadmat("/Users/wenxuan/Library/CloudStorage/Dropbox/IIT project/sub_0010-mr_0015-ecr_echo1_EEG_pp.mat")
print(mat["EEG"]["data"][0][0].shape)
EEG = mat["EEG"]["data"][0][0]
EEG_first_3_channels = EEG[:3, :10000]
print(EEG_first_3_channels.shape)


TimeoutError: [Errno 60] Operation timed out: '/Users/wenxuan/Library/CloudStorage/Dropbox/IIT project/sub_0010-mr_0015-ecr_echo1_EEG_pp.mat'

In [40]:
import pyphi
import numpy as np

EEG_first_3_channels = np.array(EEG_first_3_channels)


Welcome to PyPhi!

If you use PyPhi in your research, please cite the paper:

  Mayner WGP, Marshall W, Albantakis L, Findlay G, Marchman R, Tononi G.
  (2018). PyPhi: A toolbox for integrated information theory.
  PLOS Computational Biology 14(7): e1006343.
  https://doi.org/10.1371/journal.pcbi.1006343

Documentation is available online (or with the built-in `help()` function):
  https://pyphi.readthedocs.io

To report issues, please use the issue tracker on the GitHub repository:
  https://github.com/wmayner/pyphi

For general discussion, you are welcome to join the pyphi-users group:
  https://groups.google.com/forum/#!forum/pyphi-users

To suppress this message, either:
  - Set `WELCOME_OFF: true` in your `pyphi_config.yml` file, or
  - Set the environment variable PYPHI_WELCOME_OFF to any value in your shell:
        export PYPHI_WELCOME_OFF='yes'



In [41]:
# Calculate the Z-score for each channel
mean = np.mean(EEG_first_3_channels, axis=1)
std = np.std(EEG_first_3_channels, axis=1)

z_score = (EEG_first_3_channels - mean[:, np.newaxis]) / std[:, np.newaxis]

In [42]:
z_score_binary = np.where(z_score > 0, 1, 0)
z_score_binary.shape
z_score_binary = np.transpose(z_score_binary)
z_score_binary.shape
len(z_score_binary)

10000

In [8]:
import numpy as np
def tmp_generator(n):
    results = np.zeros((2**n, n))
    for i in range(2**n):
        for j in range(n):
            results[i, j] = i>>j & 1
    return results

In [9]:
tmp_generator(3)

array([[0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.],
       [0., 0., 1.],
       [1., 0., 1.],
       [0., 1., 1.],
       [1., 1., 1.]])

A discrete dynamical system is completely specified by its transition probability
matrix (TPM)

1) count each time one of the four states occurs and then compute their relative frequency. 
2) create a track record of what state occurs immediately after you took record of a state. Here, too, you want to compute frequencies (i.e., for each time that state 0 0 occurs, how often was the next state 0 0 again? How often was it 0 1? 1 0? And so on).

In [46]:
def count_states(data, states):
    count = np.zeros(len(states))
    for i in range(len(states)):
        state = states[i]
        count[i] = np.sum(np.all(data == state, axis=1))
    total = np.sum(count)
    frequency = count / total
    return count, frequency

In [44]:
count, frequency = count_states(z_score_binary, tmp_generator(3))
print(count)
print(frequency)


NameError: name 'count_states' is not defined

In [36]:
def count_and_frequency_of_state_transitions(data, states):
    """
    Calculates the state transition matrix and frequency of state transitions from the preprocessed EEG signal and possible binary combinations of states of nodes.

    Parameters:
        data (list): List of preprocessed EEG signal data points representing the state transitions of the nodes.
        states (list): List of all possible binary combinations of states of nodes.

    Returns:
        tuple: A tuple containing two numpy arrays, the transition matrix and the frequency of state transitions between the binary combinations of states of nodes.

    """
    transition_matrix = np.zeros((len(states),len(states)))
    for i in range(len(data)-1):
        current_state = data[i]
        next_state = data[i + 1]
        current_state_index = np.where((states == current_state).all(axis=1))[0][0]
        next_state_index = np.where((states == next_state).all(axis=1))[0][0]
        transition_matrix[current_state_index, next_state_index] += 1
    frequency = transition_matrix / np.sum(transition_matrix, axis=1, keepdims=True)

    # to avoid 1/0 = nan problem, assign as 0 or average probability if it nan
    # average = 1 / len(states)
    average = 0
    frequency[np.isnan(frequency)] = average
    return transition_matrix, frequency

In [45]:
# Calculate 
transition_matrix, frequency = count_and_frequency_of_state_transitions(z_score_binary, tmp_generator(3))
sbn_tpm = pyphi.convert.state_by_state2state_by_node(frequency)
sbs_tpm = pyphi.convert.state_by_node2state_by_state(sbn_tpm)
print(z_score_binary)

[[1 1 1]
 [1 1 1]
 [1 1 1]
 ...
 [1 1 1]
 [1 1 1]
 [1 1 1]]


In [ ]:
if __name__ == '__main__': 
    network = pyphi.Network(sbs_tpm)
    state = (1, 0, 0)
    subsystem = pyphi.Subsystem(network, state)
    result = pyphi.compute.phi(subsystem)
    print(result)


In [2]:
import numpy as np

ABC = np.array([[1,0,1,0,1,0],
[1,1,1,1,1,1],
[0,0,0,0,0,0]])

def count_consecutive_zero_pairs(A):
    """
    Given Data Matrix A (n x t, n = # of nodes, t = time points)
    returns Matrix Result (n x n, n = # of nodes)
    Each value in result represents: when time = t, if value of certain s = 0, then number of 0s at time = t+1, for each possible states S
    for example, if n = 3 (S = {A, B, C}), then value at position [0,0] represent when time = t, number of 0s of A at time t+1 

    Example:
    ABC = np.array(
        [[1,0,1,0,1,0],
        [1,1,1,1,1,1],
        [0,0,0,0,0,0]])
    Result = array(
        [[0., 0., 2.],
        [0., 0., 0.],
        [3., 0., 5.]])
    """
    result = np.zeros((len(A), len(A)))
    for i in range(len(A)):
        for k in range(len(A)):
            count = 0
            for j in range(len(A[0])-1):
                if ABC[i, j] == 0:
                    if ABC[k, j+1] == 0:
                        count = count + 1
            result[i, k] = count
    return result

count_consecutive_zero_pairs(ABC)

array([[0., 0., 2.],
       [0., 0., 0.],
       [3., 0., 5.]])

In [67]:
import numpy as np

def count_S0_at_tplus1(matrix, condition_value):
    '''
    Calculate Conditional Probability of P(A | s)
    '''
    # Initialize a dictionary to store counts for each possible state S
    counts = {}
    
    # Get the number of states
    num_nodes = matrix.shape[0]
    possible_states = tmp_generator(num_nodes)
    num_possible_states = possible_states.shape[0]

    for i in range(num_nodes):
        for j in range(num_possible_states):
            counts[(i, tuple(possible_states[j]))] = 0

    # Iterate over each time step t
    for t in range(matrix.shape[1] - 1):
        # Get the states at time t and t+1
        S_t = tuple(matrix[:, t])
        S_tplus1 = tuple(matrix[:, t+1])

        # Iterate over each state index i and count the number of times each possible state has a value of 0 for state i in the following time step
        for i in range(num_nodes):
            if S_t[i] == condition_value:
                index = np.where((possible_states == S_tplus1).all(axis=1))[0][0]
                counts[(i, tuple(S_tplus1))] += 1

    return counts

ABC = np.array([[1,0,1,0,1,0,0,0,0,0,0,0],
[0,0,1,0,1,0,0,1,0,1,0,0],
[1,0,0,0,1,0,1,0,0,0,1,0]])

sorted(count_S0_at_tplus1(ABC, 0).items())

[((0, (0.0, 0.0, 0.0)), 2),
 ((0, (0.0, 0.0, 1.0)), 2),
 ((0, (0.0, 1.0, 0.0)), 2),
 ((0, (0.0, 1.0, 1.0)), 0),
 ((0, (1.0, 0.0, 0.0)), 0),
 ((0, (1.0, 0.0, 1.0)), 0),
 ((0, (1.0, 1.0, 0.0)), 1),
 ((0, (1.0, 1.0, 1.0)), 1),
 ((1, (0.0, 0.0, 0.0)), 2),
 ((1, (0.0, 0.0, 1.0)), 1),
 ((1, (0.0, 1.0, 0.0)), 2),
 ((1, (0.0, 1.0, 1.0)), 0),
 ((1, (1.0, 0.0, 0.0)), 0),
 ((1, (1.0, 0.0, 1.0)), 0),
 ((1, (1.0, 1.0, 0.0)), 1),
 ((1, (1.0, 1.0, 1.0)), 1),
 ((2, (0.0, 0.0, 0.0)), 2),
 ((2, (0.0, 0.0, 1.0)), 2),
 ((2, (0.0, 1.0, 0.0)), 1),
 ((2, (0.0, 1.0, 1.0)), 0),
 ((2, (1.0, 0.0, 0.0)), 0),
 ((2, (1.0, 0.0, 1.0)), 0),
 ((2, (1.0, 1.0, 0.0)), 1),
 ((2, (1.0, 1.0, 1.0)), 1)]

In [83]:
import numpy as np

def calculate_conditional_prob_distribution(frequency_dict, condition):
    # Calculate P(AB | A = 0)
    counts = {}
    total_count = 0

    for k, v in frequency_dict.items():
        if k[0] == condition:
            counts[k[1]] = counts.get(k[1], 0) + v
            total_count += v

    probabilities_dict = {k: v / total_count for k, v in counts.items()}
    prob_array = np.array(list(probabilities_dict.values()))

    return probabilities_dict, prob_array

def calculate_conditional_prob_certain_states(frequency_dict, condition, conditional_event):
    # Calculate P(A | A = 0)
    counts = {}
    total_count = 0

    for k, v in frequency_dict.items():
        if k[0] == condition and k[1][conditional_event] == 0:
            counts[0] = counts.get(0, 0) + v
            total_count += v
        if k[0] == condition and k[1][conditional_event] == 1:
            counts[1] = counts.get(1, 0) + v
            total_count += v

    probabilities = {k: v / total_count for k, v in counts.items()}
    prob_array = np.array(list(probabilities.values()))

    return probabilities, prob_array

def calculate_row_frequency_probability(AB):
    # Calculate P(N), return (N x 2)
    freq_prob = np.zeros((AB.shape[0], 2))
    for i in range(AB.shape[0]):
        freq_prob[i, 0] = np.sum(AB[i] == 0)
        freq_prob[i, 1] = np.sum(AB[i] == 1)
        freq_prob[i] = freq_prob[i] / np.sum(freq_prob[i])
    return freq_prob

def calculate_prob_multip_normalize_simple(A, B):
    # Need to change to more generalized version
    result = [A[0] * B[0], A[0] * B[1], A[1] * B[0], A[1] * B[1]]
    sum_result = sum(result)
    if sum_result == 0:
        return [0, 0, 0, 0]
    result_normalized = [x/sum_result for x in result]
    return result_normalized

In [189]:
import numpy as np
from scipy.stats import wasserstein_distance

def calculate_states(n_nodes):
    return np.array([[int(c) for c in bin(i)[2:].zfill(n_nodes)] for i in range(2**n_nodes)])

def find_indices_zeros_ones(n_nodes, k):
    '''Find in at k-th digit which element = 0 and = 1, return their indexes'''
    arr = np.arange(2**n_nodes)
    # from big endian to little endian
    k = n_nodes - k - 1
    # check if k-th bit is zero
    kth_bit_zero = np.bitwise_and(np.right_shift(arr, k), 1) == 0
    kth_bit_one = np.bitwise_and(np.right_shift(arr, k), 1) == 1
    # return indices where k-th bit is zero and one
    zeros = np.where(kth_bit_zero)[0]
    ones = np.where(kth_bit_one)[0]
    return zeros, ones

def remove_node(states, i):
    '''remove i-th node, leave remaining states'''
    n = states.shape[1]
    remaining_states = np.zeros((2**(n), n-1), dtype=int)
    for j in range(n-1):
        if j < i:
            remaining_states[:, j] = states[:, j]
        else:
            remaining_states[:, j] = states[:, j+1]
    return remaining_states

def find_same_elements(arr, detect_same_array=True):
    ''' a numpy array arr of shape (m, n) and returns a numpy array same_values of shape (n, 2) 
    where each row in same_values contains the indices of the rows in arr that have the same values in the corresponding column.
    '''
    if detect_same_array:
        unique_arr, unique_idx = np.unique(arr, axis=0, return_index=True)
        same_idxs = [[] for _ in unique_arr]
        for i, unique_elem in enumerate(unique_arr):
            for j, elem in enumerate(arr):
                if np.array_equal(unique_elem, elem):
                    same_idxs[i].append(j)
    else:
        unique_arr = np.unique(arr)
        same_idxs = []
        for elem in unique_arr:
            idxs = np.where(arr == elem)[0]
            same_idxs.append(idxs.tolist())
    return same_idxs

def assign_means_disconnected_node(input_array, index_array):
    output_array = input_array.copy()
    for indices in index_array:
        avg = np.mean(input_array[indices])
        for index in indices:
            output_array[index] = avg
    return output_array





In [235]:
# Calculate P(AB | A = 0)
AB = np.array([[1,0,1,1,1,0,0,0,1,0,1,1,1,1,0],
[0,0,1,1,1,1,1,0,1,0,1,0,0,1,1]])

# Before "Disconnection"
print("Before Disconnection")
transition_matrix, frequency = count_and_frequency_of_state_transitions(np.transpose(AB), tmp_generator(2))
print("Total Transition Matrix: \n", frequency)

# P(s | n = 0): all states given each node s = 0
frequency_dict = count_S0_at_tplus1(AB, 0)
print("Total Conditional Frequency: \n",frequency_dict)

# P(AB | A = 0)
probabilities, prob_array = calculate_conditional_prob_distribution(frequency_dict, 0)
print("P(AB | A = 0) = ", prob_array)

# P(A | A = 0)
A_cond_probabilities, A_cond_prob_array = calculate_conditional_prob_certain_states(frequency_dict, 0, 0)
print("P(A | A = 0) = ", A_cond_prob_array)

# P(B | A = 0)
B_cond_probabilities, B_cond_prob_array = calculate_conditional_prob_certain_states(frequency_dict, 0, 1)
print("P(B | A = 0) = ", B_cond_prob_array)

# P(A) & P(B)
total_frequency = calculate_row_frequency_probability(AB)
print("P(A) & P(B) = \n", total_frequency)

# P(A | A = 0) * P(B)
result_A = calculate_prob_multip_normalize_simple(A_cond_prob_array, total_frequency[1])
print("P(A | A = 0) * P(B) = ", result_A)
disconnected_average_AA = [0.25, 0.25, 0.25, 0.25]
phi_effect_A = wasserstein_distance(disconnected_average_AA, result_A)
print("phi_effect_A = ", phi_effect_A)

# P(B | A = 0) * P(A)
result_B = calculate_prob_multip_normalize_simple(B_cond_prob_array, total_frequency[0])
print("P(B | A = 0) * P(A) = ",result_B)
disconnected_average_AB = [0.25, 0.25, 0.25, 0.25]
phi_effect_B = wasserstein_distance(disconnected_average_AB, result_B)
print("phi_effect_B = ", phi_effect_B)

zeros, ones = find_indices_zeros_ones(n_nodes = 2, k = 0)

states = calculate_states(2)
print("All Possible States for 2 Nodes: \n",states)

print("\n\nCalculate average: \n")

print("Remove connection A - A: \n")
removed_states = remove_node(calculate_states(2), 0) #remove A
print("If we REMOVE Node A from All states, the remaining: \n",removed_states) 
find_same_values_index = find_same_elements(removed_states, detect_same_array=True)
print("The index of same remaining value: \n", find_same_values_index)
disconnected_average_AA = assign_means_disconnected_node(prob_array, find_same_values_index)
print("Before: P(AB | A = 0): ", prob_array)
print("After: P(B | A = 0) * P(A) with statistically loose connection A - A: ", disconnected_average_AA)
phi_effect_AA = wasserstein_distance(prob_array, disconnected_average_AA)
print("phi_effect = ", phi_effect_AA)

print("\nRemove connection A - B: \n")
removed_states = remove_node(calculate_states(2), 1) #remove B
print("If we REMOVE Node B from All states, the remaining: \n",removed_states) 
find_same_values_index = find_same_elements(removed_states, detect_same_array=True)
print("The index of same remaining value: \n", find_same_values_index)
disconnected_average_AB = assign_means_disconnected_node(prob_array, find_same_values_index)
print("Before: P(AB | A = 0): ", prob_array)
print("After: P(A | A = 0) * P(B) with statistically loose connection A - B: ", disconnected_average_AB)
phi_effect_AB = wasserstein_distance(prob_array, disconnected_average_AB)
print("phi_effect = ", phi_effect_AB)

print("\nRemove connection A - B and A - A: \n")
disconnected_average_AAB = [0.25, 0.25, 0.25, 0.25]
print("Before: P(AB | A = 0): ", prob_array)
print("After: P(A) * P(B) with statistically loose connection A-B and A-A: ", disconnected_average_AAB)
phi_effect_AAB = wasserstein_distance(prob_array, disconnected_average_AAB)
print("phi_effect = ", phi_effect_AAB)

Before Disconnection
Total Transition Matrix: 
 [[0.         0.         0.         1.        ]
 [0.33333333 0.33333333 0.         0.33333333]
 [0.5        0.         0.5        0.        ]
 [0.16666667 0.16666667 0.33333333 0.33333333]]
Total Conditional Frequency: 
 {(0, (0.0, 0.0)): 1, (0, (1.0, 0.0)): 0, (0, (0.0, 1.0)): 1, (0, (1.0, 1.0)): 3, (1, (0.0, 0.0)): 1, (1, (1.0, 0.0)): 1, (1, (0.0, 1.0)): 0, (1, (1.0, 1.0)): 4}
P(AB | A = 0) =  [0.2 0.  0.2 0.6]
P(A | A = 0) =  [0.4 0.6]
P(B | A = 0) =  [0.2 0.8]
P(A) & P(B) = 
 [[0.4 0.6]
 [0.4 0.6]]
P(A | A = 0) * P(B) =  [0.16000000000000003, 0.24, 0.24, 0.36]
phi_effect_A =  0.05499999999999999
P(B | A = 0) * P(A) =  [0.08000000000000002, 0.12, 0.32000000000000006, 0.48]
phi_effect_B =  0.15000000000000002
All Possible States for 2 Nodes: 
 [[0 0]
 [0 1]
 [1 0]
 [1 1]]


Calculate average: 

Remove connection A - A: 

If we REMOVE Node A from All states, the remaining: 
 [[0]
 [1]
 [0]
 [1]]
The index of same remaining value: 
 [[0, 2

# Calculating a cause repertoire

In [204]:
def count_and_frequency_of_state_transitions_previous(data, states):
    # result of this == transpose of count_and_frequency_of_state_transitions
    transition_matrix = np.zeros((len(states), len(states)))
    for i in range(1, len(data)):
        current_state = data[i]
        previous_state = data[i - 1]
        current_state_index = np.where((states == current_state).all(axis=1))[0][0]
        previous_state_index = np.where((states == previous_state).all(axis=1))[0][0]
        transition_matrix[current_state_index, previous_state_index] += 1
    frequency = transition_matrix / np.sum(transition_matrix, axis=1, keepdims=True)

    average = 0
    frequency[np.isnan(frequency)] = average
    return transition_matrix, frequency


In [215]:
def count_S0_at_tminus1(matrix, condition_value):
    '''
    Calculate Conditional Probability of P(A | s)
    '''
    # Initialize a dictionary to store counts for each possible state S
    counts = {}
    
    # Get the number of states
    num_nodes = matrix.shape[0]
    possible_states = tmp_generator(num_nodes)
    num_possible_states = possible_states.shape[0]

    for i in range(num_nodes):
        for j in range(num_possible_states):
            counts[(i, tuple(possible_states[j]))] = 0

    # Iterate over each time step t
    for t in range(1 , matrix.shape[1]):
        # Get the states at time t and t+1
        S_t = tuple(matrix[:, t])
        S_tminus1 = tuple(matrix[:, t-1])

        # Iterate over each state index i and count the number of times each possible state has a value of 0 for state i in the following time step
        for i in range(num_nodes):
            if S_t[i] == condition_value:
                index = np.where((possible_states == S_tminus1).all(axis=1))[0][0]
                counts[(i, tuple(S_tminus1))] += 1

    return counts

In [236]:
# Calculate P(AB(t-1) | S)
AB = np.array([[1,0,1,1,1,0,0,0,1,0,1,1,1,1,0],
[0,0,1,1,1,1,1,0,1,0,1,0,0,1,1]])

# Before "Disconnection"
transition_matrix, frequency = count_and_frequency_of_state_transitions_previous(np.transpose(AB), tmp_generator(2))
print("Total transition frequency count: \n", transition_matrix)
print("Total transition frequency matrix (current<row> x past<column>): \n", frequency)
frequency_dict = count_S0_at_tminus1(AB, 0)
print("Total Conditional Frequency: \n",frequency_dict)

# P(AB(t-1) | A = 0)
probabilities, prob_array = calculate_conditional_prob_distribution(frequency_dict, 0)
print("P(AB | A = 0) = ", prob_array)
# P(A(t-1)| A = 0)
A_cond_probabilities, A_cond_prob_array = calculate_conditional_prob_certain_states(frequency_dict, 0, 0)
print("P(A | A = 0) = ", A_cond_prob_array)
# P(B(t-1) | A = 0)
B_cond_probabilities, B_cond_prob_array = calculate_conditional_prob_certain_states(frequency_dict, 0, 1)
print("P(B | A = 0) = ", B_cond_prob_array)
# P(A) & P(B)
total_frequency = calculate_row_frequency_probability(AB)

print("P(A) & P(B) = \n", total_frequency)
# P(A(t-1) | A = 0) * P(B)
result_A = calculate_prob_multip_normalize_simple(A_cond_prob_array, total_frequency[1])
print("P(A | A = 0) * P(B) = ", result_A)
disconnected_average_AA = [0.25, 0.25, 0.25, 0.25]
phi_cause_A = wasserstein_distance(disconnected_average_AA, result_A)
print("phi_cause_A = ", phi_cause_A)

# P(B(t-1) | A = 0) * P(A)
result_B = calculate_prob_multip_normalize_simple(B_cond_prob_array, total_frequency[0])
print("P(B | A = 0) * P(A) = ",result_B)
disconnected_average_AB = [0.25, 0.25, 0.25, 0.25]
phi_cause_B = wasserstein_distance(disconnected_average_AB, result_B)
print("phi_cause_B = ", phi_cause_B)


zeros, ones = find_indices_zeros_ones(n_nodes = 2, k = 0)

states = calculate_states(2)
print("\n\n\nRemove connection A - A: \n")
removed_states = remove_node(calculate_states(2), 0) #remove A
find_same_values_index = find_same_elements(removed_states, detect_same_array=True)
disconnected_average_AA = assign_means_disconnected_node(prob_array, find_same_values_index)
print("Before: P(AB(t-1) | A = 0): ", prob_array)
print("After: P(B(t-1) | A = 0) * P(A) with statistically loose connection A - A: ", disconnected_average_AA)
phi_cause_AA = wasserstein_distance(prob_array, disconnected_average_AA)
print("phi_cause_AA = ", phi_cause_AA)

print("\nRemove connection A - B: \n")
removed_states = remove_node(calculate_states(2), 1) #remove B
find_same_values_index = find_same_elements(removed_states, detect_same_array=True)
disconnected_average_AB = assign_means_disconnected_node(prob_array, find_same_values_index)
print("Before: P(AB(t-1) | A = 0): ", prob_array)
print("After: P(A(t-1) | A = 0) * P(B) with statistically loose connection A - B: ", disconnected_average_AB)
phi_cause_AB = wasserstein_distance(prob_array, disconnected_average_AB)
print("phi_cause_AB = ", phi_cause_AB)

print("\nRemove connection A - B and A - A: \n")
disconnected_average_AAB = [0.25, 0.25, 0.25, 0.25] #remove AB
print("Before: P(AB(t-1) | A = 0): ", prob_array)
print("After: P(A) * P(B) with statistically loose connection A-B and A-A: ", disconnected_average_AAB)
phi_cause_AAB = wasserstein_distance(prob_array, disconnected_average_AAB)
print("phi_cause_AAB = ", phi_cause_AAB)



Total transition frequency count: 
 [[0. 1. 1. 1.]
 [0. 1. 0. 1.]
 [0. 0. 1. 2.]
 [3. 1. 0. 2.]]
Total transition frequency matrix (current<row> x past<column>): 
 [[0.         0.33333333 0.33333333 0.33333333]
 [0.         0.5        0.         0.5       ]
 [0.         0.         0.33333333 0.66666667]
 [0.5        0.16666667 0.         0.33333333]]
Total Conditional Frequency: 
 {(0, (0.0, 0.0)): 0, (0, (1.0, 0.0)): 1, (0, (0.0, 1.0)): 2, (0, (1.0, 1.0)): 3, (1, (0.0, 0.0)): 0, (1, (1.0, 0.0)): 2, (1, (0.0, 1.0)): 1, (1, (1.0, 1.0)): 2}
P(AB | A = 0) =  [0.         0.16666667 0.33333333 0.5       ]
P(A | A = 0) =  [0.33333333 0.66666667]
P(B | A = 0) =  [0.16666667 0.83333333]
P(A) & P(B) = 
 [[0.4 0.6]
 [0.4 0.6]]
P(A | A = 0) * P(B) =  [0.13333333333333333, 0.19999999999999998, 0.26666666666666666, 0.39999999999999997]
phi_cause_A =  0.08333333333333333
P(B | A = 0) * P(A) =  [0.06666666666666667, 0.09999999999999999, 0.33333333333333337, 0.5]
phi_cause_B =  0.16666666666666669





In [240]:
print("Maximally irreducible CAUSE repertoire (MICC)")
print("Total small-phi of cause = MAX of", phi_cause_A, "(EMD of A-A versus A-.A),", phi_cause_B, "(EMD of A-B versus A-.B),", "max of [", phi_cause_AA,",", phi_cause_AB,",", phi_cause_AAB,"](EMD of A-AB)")
MICC = np.max([phi_cause_A, phi_cause_B, np.max([phi_cause_AA,phi_cause_AB, phi_cause_AAB])])
print("Total small-phi of cause = ", MICC)

Maximally irreducible CAUSE repertoire (MICC)
Total small-phi of cause = MAX of 0.08333333333333333 (EMD of A-A versus A-.A), 0.16666666666666669 (EMD of A-B versus A-.B), max of [ 0.08333333333333334 , 0.08333333333333334 , 0.16666666666666669 ](EMD of A-AB)
Total small-phi of cause =  0.16666666666666669


In [241]:
print("Maximally irreducible EFFECT repertoire (MICE)")
print("Total small-phi of effect = MAX of", phi_effect_A, "(EMD of A-A versus A-.A),", phi_effect_B, "(EMD of A-B versus A-.B),", "max of [", phi_effect_AA,",", phi_effect_AB,",", phi_effect_AAB,"](EMD of A-AB)")
MICE = np.max([phi_effect_A, phi_effect_B, np.max([phi_effect_AA,phi_effect_AB, phi_effect_AAB])])
print("Total small-phi of effect = ", MICE)

Maximally irreducible EFFECT repertoire (MICE)
Total small-phi of effect = MAX of 0.05499999999999999 (EMD of A-A versus A-.A), 0.15000000000000002 (EMD of A-B versus A-.B), max of [ 0.15 , 0.15 , 0.175 ](EMD of A-AB)
Total small-phi of effect =  0.175


In [242]:
print("Concepts of AB: min(phi cause, phi effect) = ", min(MICE, MICC) )

Concepts of AB: min(phi cause, phi effect) =  0.16666666666666669
